In [1]:
import nibabel as nib

In [2]:
!pwd

/mnt/e/HCP_MEG_fMRI/test


In [8]:
!ls ../data/100307/MNINonLinear/Results/rfMRI_REST1_LR

100307_rfMRI_REST1_LR_ICA_Classification_dualscreen.scene
100307_rfMRI_REST1_LR_ICA_Classification_singlescreen.scene
Atlas_hp_preclean.dtseries.nii
Movement_AbsoluteRMS.txt
Movement_AbsoluteRMS_mean.txt
Movement_Regressors.txt
Movement_Regressors_dt.txt
Movement_RelativeRMS.txt
Movement_RelativeRMS_mean.txt
PhaseOne_gdc_dc.nii.gz
PhaseTwo_gdc_dc.nii.gz
ReclassifyAsNoise.txt
ReclassifyAsSignal.txt
RestingStateStats
RibbonVolumeToSurfaceMapping
SBRef_dc.nii.gz
brainmask_fs.2.nii.gz
rfMRI_REST1_LR.L.native.func.gii
rfMRI_REST1_LR.R.native.func.gii
rfMRI_REST1_LR.nii.gz
rfMRI_REST1_LR_Atlas.dtseries.nii
rfMRI_REST1_LR_Atlas_MSMAll.dtseries.nii
rfMRI_REST1_LR_Atlas_MSMAll_hp2000_clean.dtseries.nii
rfMRI_REST1_LR_Atlas_hp2000_clean.dtseries.nii
rfMRI_REST1_LR_Atlas_hp2000_clean_bias.dscalar.nii
rfMRI_REST1_LR_Atlas_hp2000_clean_vn.dscalar.nii
rfMRI_REST1_LR_Atlas_stats.dscalar.nii
rfMRI_REST1_LR_Atlas_stats.txt
rfMRI_REST1_LR_CSF.txt
rfMRI_REST1_LR_Jacobian.nii.gz
rfMRI_REST1_LR_Physio_log.

In [14]:
!ls ../data/100307/MNINonLinear/fsaverage_LR32k

100307.32k_fs_LR.wb.spec
100307.ArealDistortion_FS.32k_fs_LR.dscalar.nii
100307.ArealDistortion_MSMAll.32k_fs_LR.dscalar.nii
100307.ArealDistortion_MSMSulc.32k_fs_LR.dscalar.nii
100307.BA.32k_fs_LR.dlabel.nii
100307.BiasField_MSMAll.32k_fs_LR.dscalar.nii
100307.EdgeDistortion_MSMAll.32k_fs_LR.dscalar.nii
100307.L.ArealDistortion_FS.32k_fs_LR.shape.gii
100307.L.ArealDistortion_MSMSulc.32k_fs_LR.shape.gii
100307.L.BA.32k_fs_LR.label.gii
100307.L.MyelinMap.32k_fs_LR.func.gii
100307.L.MyelinMap_BC.32k_fs_LR.func.gii
100307.L.SmoothedMyelinMap.32k_fs_LR.func.gii
100307.L.SmoothedMyelinMap_BC.32k_fs_LR.func.gii
100307.L.aparc.32k_fs_LR.label.gii
100307.L.aparc.a2009s.32k_fs_LR.label.gii
100307.L.atlasroi.32k_fs_LR.shape.gii
100307.L.corrThickness.32k_fs_LR.shape.gii
100307.L.curvature.32k_fs_LR.shape.gii
100307.L.flat.32k_fs_LR.surf.gii
100307.L.inflated.32k_fs_LR.surf.gii
100307.L.inflated_MSMAll.32k_fs_LR.surf.gii
100307.L.midthickness.32k_fs_LR.surf.gii
100307.L.midthickness_MSMAll.32k_fs

In [25]:
!ls ../output/

test.nii  test.pconn.nii  test.ptseries.nii


In [26]:
data_path          = "../data/100307/MNINonLinear/Results/rfMRI_REST1_LR/rfMRI_REST1_LR_Atlas_MSMAll_hp2000_clean.dtseries.nii"
parcellation_path  = "../data/100307/MNINonLinear/fsaverage_LR32k/100307.aparc.32k_fs_LR.dlabel.nii"
output_path        = "../output/test.ptseries.nii"

In [27]:
test_data = nib.load(output_path)

In [28]:
test_data.get_fdata().shape

(1200, 70)

In [16]:
parcellation = nib.load(parcellation_path)

In [17]:
parcellation.shape

(1, 59412)

In [21]:
parcellation_data = parcellation.get_fdata()

In [23]:
parcellation_data

memmap([[10., 29., 24., ..., 50., 50., 50.]])